In [1]:
# Agent model library
import agentpy as ap
# Visualization
import matplotlib.pyplot as plt
import IPython
# Random
from random import randint

amount_of_cars = 10
finalCars = []
north_positions = {}
west_positions = {}
east_positions = {}
south_positions = {}

for car in range(amount_of_cars):
    selected_index = randint(0,3)
    #north
    if selected_index == 0:
        new_position = (randint(0,30),40)
        while new_position in north_positions:
            new_position = (randint(0,30),40)
        finalCars.append(new_position)
    #west
    elif selected_index == 1:
        new_position =  (40,randint(0,30))
        while new_position in west_positions:
            new_position = (40,randint(0,30))
        finalCars.append(new_position)
    #south
    elif selected_index == 2:
        new_position =  (randint(59,89),44)
        while new_position in south_positions:
            new_position = (randint(59,89),44)
        finalCars.append(new_position)
    #east
    elif selected_index == 3:
        new_position =  (44,randint(59,89))
        while new_position in south_positions:
            new_position = (44,randint(59,89))
        finalCars.append(new_position)


#cars_beginning = [ (89,  44), #east ->
#                   ( 0,  40), #west
#                   (44,  89), #north -> fijo 34
#                   (40,  0 ), #south 
#                   (30, 40)] 

In [2]:
# Create grass
# x = 0 -> x = 30
# x = 60 -> x = 89
# y = 0 -> y = 30
# y = 60 -> y = 89

grass_positions = []

#First quarter
for i in range(30): # x axis iteration
    for j in range(30): # y axis iteration
        grass_positions.append((i, j)) # Filling the array of the grass agent positions

#Second quarter
for i in range(60, 90):
    for j in range(30):
        grass_positions.append((i,j))

#Third quarter
for i in range(30):
    for j in range(60, 90):
        grass_positions.append((i,j))

#Last quarter
for i in range(60, 90):
    for j in range(60, 90):
        grass_positions.append((i,j))




In [3]:
class junctionModel(ap.Model):
    def setup(self):
        # Create agents (cars and sidewalks)
        n_cars = int(self.p.cars)
        cars = self.agents = ap.AgentList(self, n_cars)
        
        n_grass = int((30**2)*4)
        grass = ap.AgentList(self, n_grass)
        self.agents.extend(grass)

        self.ground = ap.Grid(self, (self.p.width, self.p.height), track_empty = True, check_border = True)
        self.ground.add_agents(cars, (finalCars), empty = False)
        self.ground.add_agents(grass, (grass_positions), empty = False)

        for car in self.agents:
            car_x, car_y = self.ground.positions[car]
            # To know where the car is going to move
            if (car_x <= 89 and car_y == 44):
                car.status = 0
            if (car_x >= 0 and car_y == 40):
                car.status = 1
            if (car_x == 44 and car_y <= 89):
                car.status = 2            
            if (car_x == 40 and car_y >= 0):
                car.status = 3

        grass.status = 4
        
    def step(self):
        
        moving_cars = self.agents
        #          x   y
        west  = ( -1,  0 )
        east  = (  1,  0 )
        north = (  0, -1 )
        south = (  0,  1 )

        for car in moving_cars:
            if (car.status == 0):
                self.ground.move_by(car, west)
            if (car.status == 1):
                self.ground.move_by(car, east)
            if (car.status == 2):
                self.ground.move_by(car, north)
            if( car.status == 3):
                self.ground.move_by(car, south)

    def end(self):
        self.report('Simulacion terminada')


In [4]:
parameters = {
# Simulation steps
'cars': amount_of_cars,
'width': 90,
'height': 90,
'steps': 100,
}

In [5]:
def animation_plot(model, ax):
    attr_grid = model.ground.attr_grid('status')
    color_dict = {0:'#ff0000', 1: '#32a852' ,2: '#329ea8' , 3: '#8332a8',4: '#aacd00', None: '#ffffff'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a junction")

In [6]:
fig, ax = plt.subplots()
model = junctionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))